In [59]:
! pip3 install -qU langchain-upstage

In [60]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [61]:
import warnings

warnings.filterwarnings("ignore")

In [111]:
example_text = "헤어지는 마당에 뭘 이런 거까지 알리나 이를테면 그녀에게 흔들린 것 그건 정말이지 TMI"

In [110]:
example_text = "마지막 인사는 하지 마... 중꺾마."

In [84]:
example_text = "억까하지마, 이건 웃안웃이야!"

In [88]:
example_text = "더 이상 만나지 말라고 이건 킹리적 갓심이라고! 넌 갈비야!"

In [98]:
example_text = "우리 언니가 말없이 폰 붙들고 그딴 시답지 않은 개소리나 듣고 있길래 킹받아서 쫓아왔어요. 마누라 친구랑 하룻밤 재미봐놓고 들통나니 어쩔티비 내배째라? 언닌 그쪽 식구들한테 별의별 짓 다 당하면서 견디고 있는데, 이렇게 애매한 스탠스가 얼마나 사람 마음 힘들게 하는지 아세요? 욕 먹는 게 겁나요? 욕 한 바가지 드신다고 안 죽습니다! 사랑은 적어도 미안하단 말 하지 않는 게 기본 상식이에요. 이딴 것도 사랑이라고. 할많하않입니다 진짜."

In [115]:
example_text = "알잘딱깔센하게 처리합시다."

In [ ]:
example_text = """
어제 친구랑 만나서 노잼 영화를 봤는데, 차라리 집에서 넷플릭스나 볼 걸 그랬어.
오늘 회사에서 팀장님께 혼나서 멘붕이 왔어.
이번 시험은 거의 망했어. 내가 공부할 땐 멍청하게도 뇌피셜로만 외웠거든.
주말에 친구들이랑 만나서 불금을 즐겼어.
저 사람은 언제 봐도 핵인싸야. 어딜 가나 인기 많아.
그 사람은 너무 대박이야! 모든 일을 척척 해내는 걸 보면 존경스러워.
이번 주말에는 집콕하면서 푹 쉬기로 했어.
친구가 요즘 완전 핵노잼이라 만나도 재미가 없어.
매일매일이 너무 힘들어서 힐링이 필요해.
그 가게 음식은 솔직히 비추야. 맛도 없고 서비스도 별로였어.
"""

In [116]:
# Quick hello world
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage
llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    내가 드라마 대사를 줄테니 단어 하나마다 신조어로 추정되는지 되지 않는지 찾아서 나열해봐. 
    나열하는 방법은 단어1이 신조어라면 (단어1: 신조어 맞음) 단어2가 신조어가 아니라면 (단어2: 신조어 아님)로 쭉 나열해줘.
    ---------------------------------
    {content}.
    """
)

prompt_template.format(content="{text}")
chain = prompt_template | llm | StrOutputParser()
find_new_words_result = chain.invoke({"content": {example_text}})

In [117]:
prompt_template = PromptTemplate.from_template(
    """
    내가 (단어1: 신조어 맞음, 단어2: 신조어 아님) 으로 구성되어 있는 데이터를 줄테니 신조어 맞음에 해당되는 단어들만 찾아서 뜻을 작성해줘
    뜻을 찾아서 {{단어1: 뜻, 단어2: 뜻}}과 같은 딕셔너리 형태로 대답해.   
    ---------------------------------
    {content}.
    """
)

prompt_template.format(content="{find_new_words_result}")
chain = prompt_template | llm | StrOutputParser()
new_words_meaning = chain.invoke({"content": {find_new_words_result}})
print(new_words_meaning)

{'알잘딱깔센': '알아서 잘 딱 깔끔하고 센스있게', '처리합시다': '처리합시다'}


In [118]:
prompt_template = PromptTemplate.from_template(
    """
    내가 텍스트와 텍스트 속 바꿀 단어와 각 단어의 뜻을 줄테니, 단어 뜻을 참고해 문장을 크게 바꾸지 않고 바꿀 단어만 자연스럽게 문장을 수정해서 보여줘.
    ---------------------------------
    텍스트 : {content}
    텍스트 속 바꿀 단어와 뜻: {new_words_meaning}
    """
)

# prompt_template.format(content="{find_new_words_result}", new_words_meaning="{new_words_meaning}")
chain = prompt_template | llm | StrOutputParser()
result2 = chain.invoke({"content": {example_text}, "new_words_meaning":{new_words_meaning}})
print(result2)

알아서 잘 딱 깔끔하고 센스있게 처리합시다.
